In [1]:
import pandas as pd
import numpy as np
import polars as pl

In [2]:
df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
df_test = pd.read_csv("/root/kag_comp/UM_MCTS/Data/test.csv")
df_concepts = pd.read_csv("/root/kag_comp/UM_MCTS/Data/concepts.csv")

In [3]:
def checking_set(col, start, end) :
    """
    포함하는 계층형 구조를 지닌 행들을 식별할 때 사용
    """
    lst = list(df_train.columns)[start:end]
    if (df_train.loc[df_train[col] == 1,].iloc[:, start:end] == pd.Series({c:0 for c in lst})).all(axis = 1).sum() == 0 :
        print("X = 1 : 하나라도 1인 게 있음(문제 없음)")
    else :
        print("X = 1 : 전부 0인 게 있음(문제 있음)")

    if (df_train.loc[df_train[col] == 0,].iloc[:, start:end] == pd.Series({c:1 for c in lst})).sum().sum() == 0 :
        print("X = 0 : 전부 0임(문제 없음)")
    else :
        print("X = 0 : 하나라도 1인게 있음(문제 있음)")

In [4]:
lst = []

for i, l in enumerate(df_train) :
    lst.append(df_train[l].unique())

In [5]:
lst2 = []

for i, l in enumerate(df_train) :
    lst2.append(list(df_train[[l]].groupby(l).agg({l : 'count'})[l]))

In [6]:
set(df_concepts.Name) - set(df_train.columns)

{'SkillTrace',
 'SkillTraceErrorIntercept',
 'SkillTraceErrorSlope',
 'SkillTraceScore',
 'SkillTraceTrials'}

**피쳐 설명이 엇갈리는 부분이 있으므로, (https://ludii.games/searchConcepts.php) 를 참고할 것**

In [7]:
df_train.head()

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333


In [8]:
df_train.columns

Index(['Id', 'GameRulesetName', 'agent1', 'agent2', 'Properties', 'Format',
       'Time', 'Discrete', 'Realtime', 'Turns',
       ...
       'DoLudeme', 'Trigger', 'PlayoutsPerSecond', 'MovesPerSecond',
       'EnglishRules', 'LudRules', 'num_wins_agent1', 'num_draws_agent1',
       'num_losses_agent1', 'utility_agent1'],
      dtype='object', length=814)

In [10]:
df_test.iloc[:, [1,2,3]+features_list+list(range(120, 810))]

NameError: name 'features_list' is not defined

In [11]:
features_list = [12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
                 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
                 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
                 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121))

In [12]:
df_train.iloc[:,[1,2,3] + [12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121)) + list(range(120, 810))]

,GameRulesetName,agent1,agent2,Stochastic,AsymmetricPiecesType,PlayersWithDirections,Team,SquareShape,HexShape,TriangleShape,...,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules
0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,0,0,0,0,0,0,0,...,1,0,1,0,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."
233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."
233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."
233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,0,0,0,0,1,0,0,...,1,0,1,1,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {..."


In [13]:
df_features = pd.DataFrame({'uniqueset' : lst, 'colname' : df_train.columns, 'description' : [' ', ' ', ' ', ' '] + list(df_concepts.iloc[:804,].Description) + [' ',' ',' ', ' ', ' ', ' '], 'LeafNode' : [' ', ' ', ' ', ' '] + list(df_concepts.iloc[:804,].LeafNode) + [' ',' ',' ', ' ', ' ', ' '], 'unique_count' : lst2})

**특별한 표기가 없다면 반응변수는 전부 바이너리 변수**

Stochastic : 게임에 확률적 요소가 포함됩니다.

Asymmetric : 규칙 및 파워의 비대칭성이 존재합니다.

> AsymmetricForces : 플레이어의 파워가 다릅니다.
>
> > AsymmetricPiecesType : 각 플레이어가 보유한 말의 종류가 다릅니다. (리프 노드)

PlayersWithDirections : 플레이어들이 방향을 사용합니다.

Cooperation : 플레이어들은 협력해야 합니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> Team : 게임에는 플레이어 팀이 포함됩니다.

(38) Shape : 게임판의 모양이 존재합니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> SpiralShape : 나선형 보드입니다.
>
> StarShape : 별 모양 보드입니다.

(54) Tiling : 보드는 타일로 이뤄집니다.

> **관측수가 적고 의미가 적어보이는 리프 노드**
>
> SpiralTiling : 나선형 타일입니다.

(67) MancalaBoard : 만칼라 형태의 게임이다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(74) AlquerqueBoard : 앨커키 형태의 게임이다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(79) ThreeMensMorrisBoard : 쓰리 맨즈 모리스 보드이다.

(87) Track : 보드에 트랙이 존재한다. **(하위 노드의 값이 전부 0인 행이 존재한다.)**

(93) PlayableSites : 플레이 가능한 구역

> (97) NumPlayableSitesOnBoard : 보드 위에 사용가능한 위치의 수. 여기부터 수치형이 존재한다.

(98) NumColumns : 보드는 n개의 열로 구성되어 있다. ~ (120) NumPhasesBoard : 보드 변화 페이즈

(121) Hand : 플레이어가 손에 기물을 가질 수 있다.

(122) NumContainers : 저장공간은 n개가 존재한다.

(123) NumPlayableSites : 게임에서 전체 사용 가능한 장소가 n개 존재한다. (제거 가능)

(125) Piece : 게임에서 말을 사용합니다.

(126) PieceValue : 말이 값을 가집니다.

(127) PieceRotation : 말이 회전할 수 있습니다.

(128) PieceDirection : 말이 이동방향을 가집니다.

(129) Dice : 주사위를 사용합니다.

> DiceD2 : 동전을 사용합니다.
>
> DiceD4 : 4면체 주사위가 사용됩니다.
>
> DiceD6 : 6면체 주사위가 사용됩니다.

(137) LargePiece : 게임에서 큰 말을 사용합니다.

(138) Tile : 게임에서 타일을 사용합니다.

(139) NumComponentsType : 게임에 총 n개의 구성요소가 포함됩니다.

(140) NumComponentsTypePerPlayer : 각 플레이어는 n개의 구성요소를 소유합니다. - CV 필요, 사실상 139번과 같은 변수

(141) NumDice : 주사위의 개수

(143) Meta : 특정 조건에서 최우선시되는 전역적 규칙이 존재합니다.

> SwapOption : 두 번째 플레이어가 색상을 바꿀 수 있는 선택지를 가집니다.
>
> TurnKo : 단일 턴 내에서 동일한 말의 위치를 반복할 수 없습니다.
>
> PositionalSuperko : 게임 전체에서 동일한 말의 위치가 반복될 수 없습니다.
>
> AutoMove : 특정 위치와 관련된 모든 움직임을 자동으로 적용합니다.

(152) Start : 시작 규칙

> PiecesPlacedOnBoard : 게임 시작 시 보드 위에 일부 말을 배치합니다.
>
> PiecesPlacedOutsideBoard : 게임 시작 시 보드 외부에 일부 말을 배치합니다.
>
> InitialRandomPlacement : 게임 시작 시 일부 말을 무작위로 배치합니다.
>
> InitialScore : 게임 시작 시 초기 점수를 세팅합니다.
>
> InitialCost : 게임 시작 시 계량 요소에 초기 비용을 세팅합니다.
>
> NumStartComponentsBoard : 시작 시 보드 위에 놓인 개체의 총 수
>
> NumStartComponentsHand : 시작 시 플레이어가 소지한 개체의 총 수

(169) NoSiteMoves : 플레이어가 특정 위치와 관련 없는 행동을 합니다.

> VoteDecision : 플레이어가 투표를 진행합니다. (제거함)
>
> SwapPlayersDecision : 플레이어가 순서를 교환할 수 있습니다.
>
> > SwapPlayersDecisionFrequency : 순서 교환 발생의 빈도
>
> PassDecision : 플레이어가 턴을 넘길 수 있습니다.
>
> > PassDecisionFrequency : 턴 패스 발생의 빈도

In [ ]:
## 애매한 변수들
[172, 174, 175, 178] # 애매하면 그냥 넣고 CV하는 게 좋음.
## categorical로 하면 좋을 것 같은 변수들
[181]

In [ ]:
[12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121)) + [121,
 122, 123, 125, 126, 127, 128, 129, 130, 132, 133, 137, 138,
 139, 141, 143, 145, 147, 149, 151, 153, 154, 155, 156, 159,
 160, 161, 169, 172, 174, 175, 178, 179, 180, 181, 
 ]

In [128]:
df_train.utility_agent1.describe()
df_train.loc[df_train.ProposeDecisionFrequency == 0.05].utility_agent1.describe()

count    174.000000
mean       0.639847
std        0.443083
min       -0.733333
25%        0.466667
50%        0.866667
75%        1.000000
max        1.000000
Name: utility_agent1, dtype: float64

In [ ]:
# print(df_train.utility_agent1.mean())
# print(df_train.loc[df_train.AlquerqueBoardWithEightTriangles == 1,].utility_agent1.mean())
checking_set('Moves', 168, )
# 0 in df_train.NumPlayableSites.unique()

X = 1 : 하나라도 1인 게 있음(문제 없음)
X = 0 : 전부 0임(문제 없음)


In [115]:
((df_train.PassDecisionFrequency == 0.0) & (df_train.PassDecision == 1)).sum()

4554

In [107]:
(df_train.PassDecisionFrequency == 0.0).sum()

142558

In [129]:
df_features.iloc[182:200, ]

,uniqueset,colname,description,LeafNode,unique_count
182,"[1, 0]",SingleSiteMoves,Frequency of Slide Decision To Empty move.,1,"[164092, 69142]"
183,"[1, 0]",AddDecision,Slide move.,0,"[197586, 35648]"
184,"[0.99, 0.0, 1.0, 0.56, 0.55, 0.25, 0.91, 0.95,...",AddDecisionFrequency,Frequency of Slide Decision To Enemy move.,1,"[197446, 272, 186, 174, 380, 368, 360, 198, 18..."
185,"[0, 1]",PromotionDecision,Slide move.,0,"[223494, 9740]"
186,"[0.0, 0.01, 0.03, 0.02, 0.11, 0.05, 0.04]",PromotionDecisionFrequency,Frequency of Slide Decision To Friend move.,1,"[227056, 4222, 356, 844, 186, 378, 192]"
187,"[0, 1]",RemoveDecision,Decide to leap.,0,"[207600, 25634]"
188,"[0.0, 0.1, 0.08, 0.04, 0.06, 0.07, 0.05, 0.09,...",RemoveDecisionFrequency,Frequency of Leap Decision Decision move.,1,"[209392, 1430, 940, 1496, 2714, 2134, 2720, 25..."
189,"[0, 1]",RotationDecision,Decide to leap to an empty site.,0,"[233072, 162]"
190,"[0.0, 0.49]",RotationDecisionFrequency,Frequency of Leap Decision To Empty move.,1,"[233072, 162]"
191,"[0, 1]",TwoSitesMoves,Decide to leap to a friend piece.,0,"[61276, 171958]"


> 그냥 이정도까지만 하고 CV해서 변수선택 할까?